### Imports

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

### Reading the data


In [10]:
csv_path = "Data/student-por.xls"
df = pd.read_csv(csv_path)

### First look at data